# Методы восстановления регрессии

Задачу обучения по прецедентам при $Y=\mathbb{R}$ принято называть задачей восстановления регрессии. Задано пространство объектов X и множество возможных ответов Y. Существует неизвестная целевая зависимость $y^*:X\rightarrow Y$ , значения которой известны только на объектах обучающей выборки $X^\ell = (x_i, y_i)_{i-1}^\ell, y_i = y^* (x_i)$. Требуется построить алгоритм, который в данной задаче принято называть "функцией регрессии" $a: X^* \rightarrow Y$ , аппроксимирующий целевую зависимость $y^*$.

 ## Линейная регрессия
 
Для начала настроим окружение jupiter notebook:

In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

И установим внешние зависимости:

In [2]:
import numpy as np
import math
import matplotlib.pyplot as plot

plot.rcParams['figure.figsize'] = (15, 6)

Обучающую выборку возьмём из `sklearn` – `datasets.diabetes`. Выборка специально подготовлена для демонстрации возможностей линейной регрессии, содержит 442 объекта по 10 признаков.

**Задача** этого примера реализовать линейную регрессию и технику *svd (singular value decomposition)*, сравнить их между собой и сравнить с реализацией линейной регрессии библиотекой `sklearn`.

In [3]:
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error

Подготовим обучающие и тестовые выборки диабетов:

In [9]:
diabetes = datasets.load_diabetes()  # 442 Объекта, 10 признаков

def prepare_dataset(size, features):
    diabetes_X = diabetes.data[:(size * 2), :features]
    diabetes_y = diabetes.target[:(size * 2)]

    diabetes_X_train = diabetes_X[:size]
    diabetes_X_test = diabetes_X[-size:]

    diabetes_y_train = diabetes_y[:size]
    diabetes_y_test = diabetes_y[-size:]

    return diabetes_X_train, diabetes_y_train,\
           diabetes_X_test, diabetes_y_test

Линейную регрессию оформем в виде класса с методами `fit` для обучения алгоритма, методом `predict` для ответов по тестовой выборке и геттером `coeffs` для получения коэффициентов (весов) регрессии.

### Решение нормальной системы

*Линейной моделью регресси называется уравнение вида* – $\phi(x, \alpha)=\sum_{i=1}^n \alpha_j \cdot f_j(x)$. $F$ – матрица признаков. Задача – вычилить параметры алгоритма ($\alpha$).

$\alpha^* = (F^TF)^{-1}F^Ty = F^+y$

Матрица $F^+$ называется *псевдообратной*. Решив это уравнение, получим параметры алгоритма.

### Решение через SVD

Принцип *SVD* заключается в том, чтобы разложить $F$ на 3 матрицы:

$F = VDU^T$

$D$ – диагональная матрица собственных значений $F^TF$

$V$ – ортогональная матрица $l \times n$

$U$ – ортогональная матрица $n \times n$

Проделав сингулярное разбиение, можно вычислить параметры алгоритма:

$a^* = F^+y = UD^{-1}V^Ty$

### Нормализация данных

Линейная регрессия даёт очень плохие результаты, если данные перед использованием не нормировать.

Для хороших результатов, нужно центрировать каждый признак, а также ответы по среднему значению по оси координат. От этого центрирования меняются коэффициенты уравнения.

### Алгоритм

Код линейной регрессии, *svd* и нормализации данных будет находится внутри класса `LinearRegression`:

In [6]:
class LinearRegression:
    def __init__(self, svd=False, preprocess=True):
        self.__svd = svd
        self.__has_preprocess = preprocess
        self.__intercept = 0

    def fit(self, X, y):
        if self.has_preprocess:
            X, y, X_offset, y_offset = self.__preprocess(X, y)

        pseudo_inverse = self.__pseudo_svd(X) if self.is_svd else self.__pseudo_linear(X)
        self.__w = np.dot(pseudo_inverse, y)

        if self.has_preprocess:
            self.__set_intercept(X_offset, y_offset)

    def predict(self, X):
        return np.dot(X, self.__w) + self.__intercept

    @property
    def coeffs(self):
        return self.__w

    @property
    def is_svd(self):
        return self.__svd

    @property
    def has_preprocess(self):
        return self.__has_preprocess

    def __pseudo_linear(self, F):
        F_transpose = np.transpose(F)

        return np.dot(np.linalg.inv(np.dot(F_transpose, F)), F_transpose)

    def __pseudo_svd(self, F):
        P, D, Q = np.linalg.svd(F, full_matrices=False)
        V = P
        U = Q
        D = np.diag(D)

        return np.dot(np.dot(U, np.linalg.inv(D)), np.transpose(V))

    def __set_intercept(self, X_offset, y_offset):
        self.__intercept = y_offset - np.dot(X_offset, self.coeffs)

    def __preprocess(self, X, y):
        X_offset = np.average(X, axis=0)
        X -= X_offset

        y_offset = np.average(y, axis=0)
        y = y - y_offset

        return X, y, X_offset, y_offset


Для сравнения 3х алгоритмов сделаем вспомогательную функцию:

In [11]:
def compare(size, features, preprocess):
    diabetes_X_train, diabetes_y_train, diabetes_X_test, diabetes_y_test = prepare_dataset(size, features)

    sk_y_est = sklearn_predict(diabetes_X_train, diabetes_y_train, diabetes_X_test, preprocess)
    print("Средняя квадратичная ошика: %.2f"
          % mean_squared_error(diabetes_y_test, sk_y_est))

    linear_y_est = linear_predict(diabetes_X_train, diabetes_y_train, diabetes_X_test, preprocess)
    print("Средняя квадратичная ошика: %.2f"
          % mean_squared_error(diabetes_y_test, linear_y_est))

    svd_y_est = svd_predict(diabetes_X_train, diabetes_y_train, diabetes_X_test, preprocess)
    print("Средняя квадратичная ошика: %.2f"
          % mean_squared_error(diabetes_y_test, svd_y_est))

    print('---------------------------------------------------------')
    print('Правильно: ', diabetes_y_test)
    print('Skykit:    ', np.floor(sk_y_est))
    print('Linear:    ', np.floor(linear_y_est))
    print('SVD:       ', np.floor(svd_y_est))

    
def sklearn_predict(X_train, y, X_test, preprocess):
    regr = linear_model.LinearRegression(fit_intercept=preprocess)
    regr.fit(X_train, y)

    print('SKYKIT: ', np.round(regr.coef_, 2))

    return regr.predict(X_test)


def linear_predict(X_train, y, X_test, preprocess):
    regr = LinearRegression(preprocess=preprocess)
    regr.fit(X_train, y)

    print('LINEAR: ', np.round(regr.coeffs, 2))

    return regr.predict(X_test)


def svd_predict(X_train, y, X_test, preprocess):
    regr = LinearRegression(svd=True, preprocess=preprocess)
    regr.fit(X_train, y)

    print('SVD: ', np.round(regr.coeffs, 2))

    return regr.predict(X_test)

### Тесты

Запустим обучение на 50 объектах на 2х признаках без нормализации данных:

In [13]:
compare(50, 2, preprocess=False)

SKYKIT:  [-515.25  333.14]
Средняя квадратичная ошика: 20454.51
LINEAR:  [-515.25  333.14]
Средняя квадратичная ошика: 20454.51
SVD:  [-515.25  333.14]
Средняя квадратичная ошика: 20454.51
---------------------------------------------------------
Правильно:  [155. 225.  59. 104. 182. 128.  52.  37. 170. 170.  61. 144.  52. 128.
  71. 163. 150.  97. 160. 178.  48. 270. 202. 111.  85.  42. 170. 200.
 252. 113. 143.  51.  52. 210.  65. 141.  55. 134.  42. 111.  98. 164.
  48.  96.  90. 162. 150. 279.  92.  83.]
Skykit:     [-33. -14.  12. -11.  10.   6.   6.  -1. -37. -16.  21.  38.  30.   2.
 -18.  40.  21.  -5.  -3. -24. -14. -14. -16.  10.  10.  21.  32.  34.
 -18.  38. -52.  10.  -7.   4. -16. -39.  53. -39.  44.   8. -22. -39.
  -5.  10.  25.  21. -13.  -1.  15. -14.]
Linear:     [-33. -14.  12. -11.  10.   6.   6.  -1. -37. -16.  21.  38.  30.   2.
 -18.  40.  21.  -5.  -3. -24. -14. -14. -16.  10.  10.  21.  32.  34.
 -18.  38. -52.  10.  -7.   4. -16. -39.  53. -39.  44.   8. -22.

Алгоритмы имеют одинаковые веса и одинаковые значения `y`, но при этом дают почти случайный результат.

Включим нормализацию данных:

In [14]:
compare(50, 2, preprocess=True)

SKYKIT:  [124.45  70.41]
Средняя квадратичная ошика: 4128.82
LINEAR:  [124.45  70.41]
Средняя квадратичная ошика: 4128.82
SVD:  [124.45  70.41]
Средняя квадратичная ошика: 4081.00
---------------------------------------------------------
Правильно:  [155. 225.  59. 104. 182. 128.  52.  37. 170. 170.  61. 144.  52. 128.
  71. 163. 150.  97. 160. 178.  48. 270. 202. 111.  85.  42. 170. 200.
 252. 113. 143.  51.  52. 210.  65. 141.  55. 134.  42. 111.  98. 164.
  48.  96.  90. 162. 150. 279.  92.  83.]
Skykit:     [144. 154. 133. 139. 134. 135. 135. 137. 145. 155. 131. 142. 143. 136.
 155. 141. 146. 152. 151. 142. 140. 140. 155. 148. 148. 146. 143. 128.
 141. 127. 149. 148. 138. 135. 140. 146. 138. 146. 140. 134. 142. 146.
 138. 134. 130. 131. 154. 137. 147. 140.]
Linear:     [144. 154. 133. 139. 134. 135. 135. 137. 145. 155. 131. 142. 143. 136.
 155. 141. 146. 152. 151. 142. 140. 140. 155. 148. 148. 146. 143. 128.
 141. 127. 149. 148. 138. 135. 140. 146. 138. 146. 140. 134. 142. 146.
 13

Нормализация улучшила ответ в несколько раз и сделала `y` похожими на настоящий ответ. Однако, значения *svd* немного отличаются. Увеличим число признаков:

In [15]:
compare(50, 3, preprocess=True)

SKYKIT:  [ -72.3  -209.53 1033.39]
Средняя квадратичная ошика: 3609.81
LINEAR:  [ -72.3  -209.53 1033.39]
Средняя квадратичная ошика: 3609.81
SVD:  [932.88 495.77  30.99]
Средняя квадратичная ошика: 5924.98
---------------------------------------------------------
Правильно:  [155. 225.  59. 104. 182. 128.  52.  37. 170. 170.  61. 144.  52. 128.
  71. 163. 150.  97. 160. 178.  48. 270. 202. 111.  85.  42. 170. 200.
 252. 113. 143.  51.  52. 210.  65. 141.  55. 134.  42. 111.  98. 164.
  48.  96.  90. 162. 150. 279.  92.  83.]
Skykit:     [146. 149. 150. 141. 186. 108. 202.  93.  87. 105. 157. 144. 131. 120.
 105. 114. 118. 118. 103. 107.  84. 191. 128. 115. 138. 105. 168. 126.
  96. 125. 164. 112. 121. 169. 115. 227.  64. 147.  98. 110. 129. 207.
 162.  93. 144. 102. 143. 250. 131.  90.]
Linear:     [146. 149. 150. 141. 186. 108. 202.  93.  87. 105. 157. 144. 131. 120.
 105. 114. 118. 118. 103. 107.  84. 191. 128. 115. 138. 105. 168. 126.
  96. 125. 164. 112. 121. 169. 115. 227.  64. 1

*svd* начал сильно отставать. Чем больше будет количество признаков, тем сильнее он будет отличаться от ближайших алгоритмов. Отставание не зависит от кличества объектов выборки, но зависит от нормализации.

Видимо, обычной нормализации по оси координат недостаточно для *svd*, чтобы показывать оптимальные результаты для `features > 2`.